In [ ]:
find . -type f -name '#*#' -delete

In [ ]:
module purge
module load GCC/10.3.0
module load OpenMPI/4.1.1
module load GROMACS/2021.3-CUDA-11.3.1

# Native

In [ ]:
WORKDIR=/gpfs0/home1/gdwanglab/axk201/personal_projects/Pediatric_glioblastoma_singelcell/Notebook/Step22_Molecular_dynamics/in/SPP1

In [ ]:
cd ${WORKDIR}

## Minimization

In [ ]:
sample=SPP1_model1
init=step3_input
mini_prefix=step4.0_minimization

prod_prefix=step5_production
prod_step=step5

In [ ]:
RUN_ID="EM"

In [ ]:
gmx grompp -f ${mini_prefix}.mdp -o ${mini_prefix}.tpr -c ${init}.gro -r ${init}.gro -p topol.top -n index.ndx -maxwarn -1

In [ ]:
./../../src/simulation_sbatch_script.sh --init_gro ${WORKDIR}/${init}.gro \
                                        --topol ${WORKDIR}/topol.top \
                                        --run_prefix ${mini_prefix} \
                                        --mdp ${WORKDIR}/step4.0_minimization.mdp \
                                        --output_dir ${WORKDIR}/../../out/${mini_prefix} \
                                        --sample ${sample} \
                                        --workdir ${WORKDIR} \
                                        --lab "gdwanglab" \
                                        --job_name "em" \
                                        --run ${RUN_ID} \
                                        --index ${WORKDIR}/index.ndx

In [ ]:
chmod 755 ${WORKDIR}/${sample}_${mini_prefix}_Run_${RUN_ID}.sbatch
sbatch ${WORKDIR}/${sample}_${mini_prefix}_Run_${RUN_ID}.sbatch

## Equilibration

In [ ]:
RUN_ID="NVT"
prod_prefix="md"
equi_prefix=step4.1_equilibration

In [ ]:
./../../src/simulation_sbatch_script.sh --init_gro ${WORKDIR}/../../out/${mini_prefix}/Run_EM/${mini_prefix}_EM.gro \
                                        --topol ${WORKDIR}/topol.top \
                                        --run_prefix ${equi_prefix} \
                                        --mdp ${WORKDIR}/step4.1_equilibration.mdp \
                                        --output_dir ${WORKDIR}/../../out/${equi_prefix} \
                                        --sample ${sample} \
                                        --workdir ${WORKDIR} \
                                        --lab "gdwanglab" \
                                        --job_name "equil" \
                                        --run_duration "80:00:00" \
                                        --run ${RUN_ID} \
                                        --index ${WORKDIR}/index.ndx

In [ ]:
sbatch ${WORKDIR}/${sample}_${equi_prefix}_Run_${RUN_ID}.sbatch

## MD

### Chain 1

In [ ]:
md_prefix="md"

In [ ]:
mkdir -p "/gpfs0/home1/gdwanglab/axk201/personal_projects/Pediatric_glioblastoma_singelcell/Notebook/Step22_Molecular_dynamics/in/SPP1/../../out/step5_production/step5_production_283"
mkdir -p "/gpfs0/home1/gdwanglab/axk201/personal_projects/Pediatric_glioblastoma_singelcell/Notebook/Step22_Molecular_dynamics/in/SPP1/../../out/step5_production/step5_production_303"
mkdir -p "/gpfs0/home1/gdwanglab/axk201/personal_projects/Pediatric_glioblastoma_singelcell/Notebook/Step22_Molecular_dynamics/in/SPP1/../../out/step5_production/step5_production_333"
mkdir -p "/gpfs0/home1/gdwanglab/axk201/personal_projects/Pediatric_glioblastoma_singelcell/Notebook/Step22_Molecular_dynamics/in/SPP1/../../out/step5_production/step5_production_353"

In [ ]:
gmx grompp -f ${WORKDIR}/step5_production_283.15.mdp \
            -o ${WORKDIR}/../../out/step5_production/step5_production_283/remd.tpr \
            -c ${WORKDIR}/../../out/${equi_prefix}/Run_NVT/${equi_prefix}_NVT.gro \
            -p ${WORKDIR}/topol.top \
            -n ${WORKDIR}/index.ndx 
            
gmx grompp -f ${WORKDIR}/step5_production_303.15.mdp \
            -o ${WORKDIR}/../../out/step5_production/step5_production_303/remd.tpr \
            -c ${WORKDIR}/../../out/${equi_prefix}/Run_NVT/${equi_prefix}_NVT.gro \
            -p ${WORKDIR}/topol.top \
            -n ${WORKDIR}/index.ndx 

gmx grompp -f ${WORKDIR}/step5_production_333.15.mdp \
            -o ${WORKDIR}/../../out/step5_production/step5_production_333/remd.tpr \
            -c ${WORKDIR}/../../out/${equi_prefix}/Run_NVT/${equi_prefix}_NVT.gro \
            -p ${WORKDIR}/topol.top \
            -n ${WORKDIR}/index.ndx 

gmx grompp -f ${WORKDIR}/step5_production_353.15.mdp \
            -o ${WORKDIR}/../../out/step5_production/step5_production_353/remd.tpr \
            -c ${WORKDIR}/../../out/${equi_prefix}/Run_NVT/${equi_prefix}_NVT.gro \
            -p ${WORKDIR}/topol.top \
            -n ${WORKDIR}/index.ndx 




In [ ]:
sbatch ${WORKDIR}/SPP1_model1_step5_production_Run_MD.sbatch

In [ ]:
grep "Repl ex" ${WORKDIR}/../../out/step5_production/step5_production_*/remd.log > ${WORKDIR}/../../out/exchange_data.log

### Center trajectories

In [ ]:
echo 0 0 | gmx trjconv -f ${WORKDIR}/../../out/step5_production/step5_production_283/remd.xtc \
            -s ${WORKDIR}/../../out/step5_production/step5_production_283/remd.tpr \
            -tu ns \
            -o ${WORKDIR}/../../out/step5_production/step5_production_283/centered.xtc \
            -pbc mol \
            -ur compact -center

echo 0 0 | gmx trjconv -f ${WORKDIR}/../../out/step5_production/step5_production_303/remd.xtc \
            -s ${WORKDIR}/../../out/step5_production/step5_production_303/remd.tpr \
            -tu ns \
            -o ${WORKDIR}/../../out/step5_production/step5_production_303/centered.xtc \
            -pbc mol \
            -ur compact -center

echo 0 0 | gmx trjconv -f ${WORKDIR}/../../out/step5_production/step5_production_333/remd.xtc \
            -s ${WORKDIR}/../../out/step5_production/step5_production_333/remd.tpr \
            -tu ns \
            -o ${WORKDIR}/../../out/step5_production/step5_production_333/centered.xtc \
            -pbc mol \
            -ur compact -center

echo 0 0 | gmx trjconv -f ${WORKDIR}/../../out/step5_production/step5_production_353/remd.xtc \
            -s ${WORKDIR}/../../out/step5_production/step5_production_353/remd.tpr \
            -tu ns \
            -o ${WORKDIR}/../../out/step5_production/step5_production_353/centered.xtc \
            -pbc mol \
            -ur compact -center

### Pull RMSD

In [ ]:
echo 0 1 | gmx rms -s ${WORKDIR}/../../out/step5_production/step5_production_283/remd.tpr \
                -f ${WORKDIR}/../../out/step5_production/step5_production_283/centered.xtc \
                -o ${WORKDIR}/../../out/step5_production/step5_production_283/rmsd.xvg \
                -fit rot+trans
echo 0 1 | gmx rms -s ${WORKDIR}/../../out/step5_production/step5_production_303/remd.tpr \
                -f ${WORKDIR}/../../out/step5_production/step5_production_303/centered.xtc \
                -o ${WORKDIR}/../../out/step5_production/step5_production_303/rmsd.xvg \
                -fit rot+trans
echo 0 1 | gmx rms -s ${WORKDIR}/../../out/step5_production/step5_production_333/remd.tpr \
                -f ${WORKDIR}/../../out/step5_production/step5_production_333/centered.xtc \
                -o ${WORKDIR}/../../out/step5_production/step5_production_333/rmsd.xvg \
                -fit rot+trans
echo 0 1 | gmx rms -s ${WORKDIR}/../../out/step5_production/step5_production_353/remd.tpr \
                -f ${WORKDIR}/../../out/step5_production/step5_production_353/centered.xtc \
                -o ${WORKDIR}/../../out/step5_production/step5_production_353/rmsd.xvg \
                -fit rot+trans

### Pull RMSF

In [ ]:
echo 1 3 | gmx rmsf -s ${WORKDIR}/../../out/step5_production/step5_production_283/remd.tpr \
                -f ${WORKDIR}/../../out/step5_production/step5_production_283/centered.xtc \
                -o ${WORKDIR}/../../out/step5_production/step5_production_283/rmsf_283.xvg -res -b 20000
echo 1 3 | gmx rmsf -s ${WORKDIR}/../../out/step5_production/step5_production_303/remd.tpr \
                -f ${WORKDIR}/../../out/step5_production/step5_production_303/centered.xtc \
                -o ${WORKDIR}/../../out/step5_production/step5_production_303/rmsf_303.xvg -res -b 20000
echo 1 3 | gmx rmsf -s ${WORKDIR}/../../out/step5_production/step5_production_333/remd.tpr \
                -f ${WORKDIR}/../../out/step5_production/step5_production_333/centered.xtc \
                -o ${WORKDIR}/../../out/step5_production/step5_production_333/rmsf_333.xvg -res -b 20000
echo 1 3 | gmx rmsf -s ${WORKDIR}/../../out/step5_production/step5_production_353/remd.tpr \
                -f ${WORKDIR}/../../out/step5_production/step5_production_353/centered.xtc \
                -o ${WORKDIR}/../../out/step5_production/step5_production_353/rmsf_353.xvg -res -b 20000

### Pull Radius of gyration

In [ ]:
echo 0 1 | gmx gyrate -s ${WORKDIR}/../../out/step5_production/step5_production_283/remd.tpr \
                      -f ${WORKDIR}/../../out/step5_production/step5_production_283/centered.xtc \
                      -o ${WORKDIR}/../../out/step5_production/step5_production_283/gyrate_283.xvg
echo 0 1 | gmx gyrate -s ${WORKDIR}/../../out/step5_production/step5_production_303/remd.tpr \
                      -f ${WORKDIR}/../../out/step5_production/step5_production_303/centered.xtc \
                      -o ${WORKDIR}/../../out/step5_production/step5_production_303/gyrate_303.xvg
echo 0 1 | gmx gyrate -s ${WORKDIR}/../../out/step5_production/step5_production_333/remd.tpr \
                      -f ${WORKDIR}/../../out/step5_production/step5_production_333/centered.xtc \
                      -o ${WORKDIR}/../../out/step5_production/step5_production_333/gyrate_333.xvg
echo 0 1 | gmx gyrate -s ${WORKDIR}/../../out/step5_production/step5_production_353/gyrate.tpr \
                      -f ${WORKDIR}/../../out/step5_production/step5_production_353/centered.xtc \
                      -o ${WORKDIR}/../../out/step5_production/step5_production_353/gyrate_353.xvg

### Energy landscape

In [ ]:
cat ${WORKDIR}/../../out/step5_production/rmsd_rg_283.dat \
    ${WORKDIR}/../../out/step5_production/rmsd_rg_303.dat \
    ${WORKDIR}/../../out/step5_production/rmsd_rg_333.dat \
    ${WORKDIR}/../../out/step5_production/rmsd_rg_353.dat > ${WORKDIR}/../../out/rmsd_rg_alltemps.dat

In [ ]:
paste <(grep -v '^[@#]' ${WORKDIR}/../../out/step5_production/step5_production_283/rmsd_283.xvg | awk '{print $2}') \
      <(grep -v '^[@#]' ${WORKDIR}/../../out/step5_production/step5_production_283/gyrate_283.xvg | awk '{print $2}') \
      <(grep -v '^[@#]' ${WORKDIR}/../../out/step5_production/step5_production_283/potential_283.xvg | awk '{print $2}') | \
      awk '{print NR, $1, $2, $3}' > ${WORKDIR}/../../out/step5_production/rmsd_rg_283.dat

paste <(grep -v '^[@#]' ${WORKDIR}/../../out/step5_production/step5_production_303/rmsd_303.xvg | awk '{print $2}') \
      <(grep -v '^[@#]' ${WORKDIR}/../../out/step5_production/step5_production_303/gyrate_303.xvg | awk '{print $2}') \
      <(grep -v '^[@#]' ${WORKDIR}/../../out/step5_production/step5_production_303/potential_303.xvg | awk '{print $2}') | \
      awk '{print NR, $1, $2, $3}' > ${WORKDIR}/../../out/step5_production/rmsd_rg_303.dat

paste <(grep -v '^[@#]' ${WORKDIR}/../../out/step5_production/step5_production_333/rmsd_333.xvg | awk '{print $2}') \
      <(grep -v '^[@#]' ${WORKDIR}/../../out/step5_production/step5_production_333/gyrate_333.xvg | awk '{print $2}') \
      <(grep -v '^[@#]' ${WORKDIR}/../../out/step5_production/step5_production_333/potential_333.xvg | awk '{print $2}') | \
      awk '{print NR, $1, $2, $3}' > ${WORKDIR}/../../out/step5_production/rmsd_rg_333.dat

paste <(grep -v '^[@#]' ${WORKDIR}/../../out/step5_production/step5_production_353/rmsd_353.xvg | awk '{print $2}') \
      <(grep -v '^[@#]' ${WORKDIR}/../../out/step5_production/step5_production_353/gyrate_353.xvg | awk '{print $2}') \
      <(grep -v '^[@#]' ${WORKDIR}/../../out/step5_production/step5_production_353/potential_353.xvg | awk '{print $2}') | \
      awk '{print NR, $1, $2, $3}' > ${WORKDIR}/../../out/step5_production/rmsd_rg_353.dat


In [ ]:
cat ${WORKDIR}/../../out/step5_production/rmsd_rg_*.dat > ${WORKDIR}/../../out/rmsd_rg_all.dat

In [ ]:
python ../../src/energy_landscape.py

In [ ]:
echo 1 | gmx trjconv -s ${WORKDIR}/../../out/step5_production/step5_production_303/remd.tpr \
                    -f ${WORKDIR}/../../out/step5_production/step5_production_303/centered.xtc \
                    -o ${WORKDIR}/../../out/step5_production/minima1.pdb \
                    -dump 1655

In [ ]:
echo 1 | gmx trjconv -s ${WORKDIR}/../../out/step5_production/step5_production_333/remd.tpr \
                    -f ${WORKDIR}/../../out/step5_production/step5_production_333/centered.xtc \
                    -o ${WORKDIR}/../../out/step5_production/minima2.pdb \
                    -dump 1968